In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import time
import pyodbc
import os
import itertools
import json
import re
from json import JSONDecoder
from sc_header import createDriver,aplanar_lista,extract_json_objects

In [3]:
def scrape_url(pos_aux,tipo_aux,url_aux,precio_aux,precio_dto_aux,pagina_aux):    
    
    lista_auxiliar = []
    soup = BeautifulSoup(requests.get(url_aux).text,'html.parser')
    try:
        color = soup.find(class_='_colorName').text
    except:
        return [-1,1,-1,-1,-1,-1,-1,-1,-1,-1]
    
    id_producto = soup.find('html').get('id')
    lista_auxiliar.append([pos_aux,
                           id_producto, #ID PRODUCTO
                           tipo_aux,
                           color, #COLOR
                           soup.find('title').text.split('|')[0].strip(), #DESC
                           precio_aux,
                           precio_dto_aux,
                           soup.find(class_='_seoImg main-image').get('href'), #IMG
                           url_aux,#URL
                           pagina_aux,
    ])
    
    return lista_auxiliar    

#PAGINA_SCRAPER

def scrape_batch(url_chunk):
    chunk_resp = []
    for url in url_chunk:
        chunk_resp.append(scrape_url(url[0],url[1],url[4],url[3],url[2],url[5]))
                                    
    return chunk_resp


In [4]:
url_base = 'https://www.zara.com/mx/es/mujer-nuevo-l1180.html?v1=1445759'
url_rebajas = 'https://www.zara.com/mx/es/mujer-l1000.html?v1=693100'
fecha = datetime.date.today()

In [5]:
browser = createDriver()

In [9]:
browser.get(url_base)

In [15]:

from selenium.webdriver.common.action_chains import ActionChains


elementos_ul = browser.find_elements_by_css_selector('._category-link-wrapper.menu-item.menu-item--level-3.menu-item--is-leaf')

hover = ActionChains(browser).move_to_element(elementos_ul[0])
hover.perform()

In [29]:
browser.get(url_base)
categorias = ['COLLECTION','REBAJAS']
elementos_ul = browser.find_elements_by_css_selector('._category-link-wrapper.menu-item.menu-item--level-3.menu-item--is-leaf')
url_list = []
for link in categorias:
    
    hover = ActionChains(browser).move_to_element(elementos_ul[0])
    hover.perform()
    time.sleep(2)
    WebDriverWait(browser,50).until(EC.presence_of_element_located((By.LINK_TEXT,link))).click()

    #elementos_ul = browser.find_elements_by_css_selector('._category-link-wrapper.menu-item.menu-item--level-3.menu-item--is-leaf')
    tipos = []
    ul = browser.find_element_by_css_selector('._subcategories.subcategory-menu.subcategory-menu--level-2.subcategory-menu--current')
    lis = ul.find_elements_by_tag_name('li')

    for indices, m in enumerate(lis):
        try:
            text_aux = browser.find_element_by_css_selector('._subcategories.subcategory-menu.subcategory-menu--level-2.subcategory-menu--current').find_elements_by_tag_name('li')[indices]
        except:
            continue
        if text_aux.text != "":
            if text_aux.text not in ['ÚLTIMA SEMANA','ZAPATOS','BOLSOS','ACCESORIOS','NEWMUM','TARJETA REGALO']:
                tipo = text_aux.text
                tipos.append(tipo)

                    #browser.execute_script('history.back();')
    for element in tipos:

        try:
            if len(element.split()) > 1 :
                hola


            for _ in range(10):
                try:
                    WebDriverWait(browser,80).until(EC.element_to_be_clickable((By.LINK_TEXT,element))).click()
                    break
                except:
                    time.sleep(.2)
        except:

            for _ in range(10):
                try:
                    WebDriverWait(browser,80).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT,element.split()[0]))).click()
                    break
                except:
                    time.sleep(.2)



        url_list.append([element,browser.current_url])


In [31]:
urls = pd.DataFrame(url_list)

In [32]:
urls = urls[(urls[0]!='A PARTIR DE -60%')&
            (urls[0]!='ACCESORIOS')&
            (urls[0]!='ZAPATOS')&
            (urls[0]!='BOLSOS')
           ]

In [33]:
urls.rename(columns={0:'tipo',
                    1:'url'},
           inplace=True)

In [91]:
hrefs_list = []
for index,row in urls.iterrows():
    try:
        browser.get(row['url'])
    except:
        browser.quit()
        time.sleep(1)
        browser = createDriver2()
        continue
    
    ELEMENTS_A = browser.find_elements_by_css_selector('a.item._item')
                                                         
    soup = BeautifulSoup(browser.page_source)
    
    for indices, element in enumerate(ELEMENTS_A):
        
        for i in soup.find_all(class_='price _product-price'):
                try:
                    precio = i.find(class_='main-price').get('data-price')
                    precio_dto = precio
                except:
                    precio = i.find(class_='line-through').get('data-price')
                    precio_dto = i.find(class_='sale').get('data-price')

        hrefs_list.append([indices,row['tipo'],precio_dto,precio,element.get_attribute('href'),row['url']            
        ])
        
    

In [93]:
batch_size = 5
url_chunks = [hrefs_list[x:x+batch_size] for x in range(0, len(hrefs_list), batch_size)]
items = []

In [94]:
for url_chunk in url_chunks:
    items.append(scrape_batch(url_chunk))

In [97]:
new_list = aplanar_lista(items)

In [101]:
df = pd.DataFrame(new_list)

In [102]:
df.rename(columns={0:'pos',
                  1:'id_producto',
                  2:'tipo',
                  3:'color',
                  4:'descripcion',
                  5:'precio',
                  6:'precio_dto',
                  7:'img',
                  8:'url',
                  9:'pagina_scraper'},
          inplace=True
         )

In [106]:
# id_sc3_producto,id_sc3_corrida,marca,tipo,tipo_es,color,color_es,sexo,descripcion,moneda,precio,precio_original,fecha_alta,url_imagen,url_producto,origen,tipo_juana,grupo

In [107]:
df['marca'] ='ZARA'
df['tipo_es'] = df['tipo']
df['color_es'] = df['color']
df['sexo'] = 'Mujer'
df['moneda'] = 'PESO MXN'
df['origen'] = 'ZARA MX'

In [108]:
df['id_producto'] = df['id_producto'].apply(lambda x:x.split('-')[1])

In [109]:
df['precio'] = (df['precio']
                .str.extract(r"([\d,\.]+)", expand=False)
                .str.replace(".", "")
                .str.replace(",", ".")
                .astype(float))

In [110]:
df['precio_dto'] = (df['precio_dto']
                    .str.extract(r"([\d,\.]+)", expand=False)
                    .str.replace(".", "")
                    .str.replace(",", ".")
                    .astype(float))

In [ ]:
df.to_excel(f'zara{fecha}.xlsx')

In [ ]:
browser.quit()

In [112]:
!jupyter nbconvert --to script zara.ipynb

[NbConvertApp] Converting notebook zara.ipynb to script
[NbConvertApp] Writing 7225 bytes to zara.py
